# Example 06: Gas metering station uncertainty calculations

This notebook demonstrates uncertainty calculations for a gas metering station consisting of an Ultrasonic Flowmeter (USM), a Gas Chromatograph (GC) and dual pressure and temperature measurements. 
Evaluations are done using both standard uncertainty calculations (based on sensitivity coefficients and input uncertainties), as well as the Monte Carlo Method. 

In [25]:
# Import Required Libraries
import pvtlib
from uncertaintylib import uncertainty_functions
import pandas as pd
import os


### Read inputs from CSV file

The inputs to the uncertainty calculation is given in a csv file. 

The input contains:
- Pressure
- Temperature
- Actual volume flowrate measured by the USM
- The calibration curve for the USM
- Gas composition measured by the GC

In [26]:
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
data_path = os.path.join(notebook_dir, 'data.csv')
df = pd.read_csv(data_path)

df # Display the dataframe

,input_name,unit,mean,standard_uncertainty,standard_uncertainty_percent,distribution,min,max
0,USM_pressure_A,bara,140.00,0.150,NaN,normal,NaN,NaN
1,USM_pressure_B,bara,140.00,0.150,NaN,normal,NaN,NaN
2,USM_temperature_A,C,50.00,0.150,NaN,normal,NaN,NaN
3,USM_temperature_B,C,50.00,0.150,NaN,normal,NaN,NaN
4,USM_volflow,m3/h,18700.00,0.300,0.05,normal,NaN,NaN
5,USM_calib_volflow_01,m3/h,2332.00,NaN,0.16,normal,NaN,NaN
6,USM_calib_volflow_02,m3/h,3888.00,NaN,0.10,normal,NaN,NaN
7,USM_calib_volflow_03,m3/h,6480.00,NaN,0.09,normal,NaN,NaN
8,USM_calib_volflow_04,m3/h,10800.00,NaN,0.07,normal,NaN,NaN
9,USM_calib_volflow_05,m3/h,18000.00,NaN,0.07,normal,NaN,NaN


# Convert dataframe to dictionary
Input to uncertaintylib must be a dictionary. Convert the dataframe to a dictionary

In [27]:
df=df.set_index('input_name')

inputs = df.to_dict() # Convert dataframe to dictionary, which is the required format for uncertaintylib

### Define the function performing the gas metering station calculations

In [ ]:
def calculate_gas_metering_station(inarray):
    """
    Function to calculate the gas metering station output parameters.
    Inputs is a dictionary with all input parameters.
    Outputs is a dictionary with all output parameters.
    """

    outarray = {}   # Create an empty dictionary to store output parameters. 

    # Set up AGA8 object for GERG-2008 equation. This process runs the setup for the given equation
    gerg = pvtlib.AGA8('GERG-2008')

    composition = {
        'N2' : inarray['N2'],
        'CO2' : inarray['CO2'],
        'C1' : inarray['C1'],
        'C2' : inarray['C2'],
        'C3' : inarray['C3'],
        'iC4' : inarray['iC4'],
        'nC4' : inarray['nC4'],
    }

    outarray['USM_Pressure_used'] = (inarray['USM_pressure_A']+inarray['USM_pressure_B'])/2 # Calculate average pressure
    outarray['USM_Temperature_used'] = (inarray['USM_temperature_A']+inarray['USM_temperature_B'])/2 # Calculate average temperature

    # Calculate gas properties
    gas_properties = gerg.calculate_from_PT(
        composition=composition,
        pressure=outarray['USM_Pressure_used'],
        temperature=outarray['USM_Temperature_used'],
        pressure_unit='bara',
        temperature_unit='C',
    )

    # Store results in output dictionary
    outarray['Gas_Density'] = gas_properties['rho']  # Gas density in kg/m3
    outarray['Gas_Compressibility'] = gas_properties['z']  # Gas compressibility factor
    outarray['Gas_Molar_Mass'] = gas_properties['mm']  # Gas molar mass in kg/kmol

    return outarray

### Calculate uncertainty using sensitivity coefficients and input uncertainties

In [29]:
uncertainty_results = uncertainty_functions.calculate_uncertainty(
    indata=inputs,
    function=calculate_gas_metering_station
)

uncertainty_results # Display the results

KeyError: 'Z'